<p>The proper divisors of a number are all the divisors excluding the number itself. For example, the proper divisors of 28 are 1, 2, 4, 7, and 14. As the sum of these divisors is equal to 28, we call it a perfect number.</p>
<p>Interestingly the sum of the proper divisors of 220 is 284 and the sum of the proper divisors of 284 is 220, forming a chain of two numbers. For this reason, 220 and 284 are called an amicable pair.</p>
<p>Perhaps less well known are longer chains. For example, starting with 12496, we form a chain of five numbers:</p>
<p class="center">12496 → 14288 → 15472 → 14536 → 14264 (→ 12496 → ...)</p>
<p>Since this chain returns to its starting point, it is called an amicable chain.</p>
<p>Find the smallest member of the longest amicable chain with no element exceeding one million.</p>


**Solution(s):**
We're searching for the longest cycle in a directed graph. We propose two solutions - one which does the work from square one, and the other which uses built in algorithms in networkx to find cycles. Both solutions use a helper function, divisorGenerator, that we've defined in previous solutions, which takes in an integer input $n$ and returns all of the divisors of $n$.

From scratch solution: We iterate through all $i \in [1, 10^6]$ and perform a depth first search on the next vertex. For some $i$, we create a list of succeeding vertices until the sum of the proper divisors is a value that we've seen before (for any $i$), or above $10^6$. If it's a value that we've seen before, we check if it's in the current list, in which case we've generated a new cycle that we haven't seen before. We find the length of that cycle and if it's the longest that we've seen so far, we save the cycle. If it's a value that we've seen in another list, then it will lead to a cycle that we've seen before. Finally, we take the minimum value of this longest cycle.

The networkx solution has clean code, where we instantiate a directed graph and add its edges. We then look at cycles in the graph and take the minimum value of the longest cycle.

### Scratch solution

In [1]:
from math import sqrt
def divisorGenerator(n):
    # A function that generates a list of divisors of $n$. 
    small_divisors = []
    large_divisors = []
    for i in range(1, int(sqrt(n) + 1)):
        if n % i == 0:
            small_divisors.append(i)
            if i*i != n:
                large_divisors.append(int(n / i))
    for divisor in reversed(large_divisors):
        small_divisors.append(divisor)
    return small_divisors


In [47]:
# Initialize the nodes and color them all "unseen" except for the first few trivial cases
nodes = [[i,0] for i in range(10**6+1)]
nodes[0] = [0,1]
nodes[1] = [0,1]
nodes[2] = [0,1]
mxLoop = []                        # stores the longest loop we've seen thus far

for i in range(3,10**6+1):
    loop = [i]
    cur = i                 # current node
    
    # We now do depth first search, stopping when a sum of divisors is too large or is a value we've seen before
    while cur <= 10**6 and nodes[cur][1] == 0:
        divSum = sum(divisorGenerator(cur))-cur
        nodes[cur][1] = divSum                         # colors the current node
        loop.append(divSum)                            # extends the current list of nodes
        if divSum in loop[:-1] and len(loop) > 2:      # checks if we've found a cycle
            
            # print("found a cycle", i, cur, loop)
            
            # next, check if its the longest one thus far 
            for j in range(len(loop)):
                if loop[j] in loop[j+1:]:
                    l = len(loop)-j
                    if l > len(mxLoop):
                        mxLoop = loop[j+1:]
                        print("New longest loop", mxLoop)
                    break

        # Here, we continue the DFS
        else:
            cur = divSum

New longest loop [284, 220]
New longest loop [1210, 1184]
New longest loop [2620, 2924]
New longest loop [5564, 5020]
New longest loop [19116, 31704, 47616, 83328, 177792, 295488, 629072, 589786, 294896, 358336, 418904, 366556, 274924, 275444, 243760, 376736, 381028, 285778, 152990, 122410, 97946, 48976, 45946, 22976, 22744, 19916, 17716, 14316]


In [41]:
min(mxLoop)

14316

### networkx solution

In [42]:
# create the list of $d(n), paired with n$
sms = [(i, sum(divisorGenerator(i))-i) for i in range(1,10**6+1)]

In [43]:
import networkx as nx

# Create Directed Graph
G=nx.DiGraph()

# Add a list of nodes:
G.add_nodes_from([i for i in range(10**6+1)])

# Add a list of edges:
G.add_edges_from(sms)

In [49]:
longCycles = list([a for a in nx.simple_cycles(G) if len(a) >= 3])

In [50]:
min(longCycles[1])

14316